This notebook implements the code from [backtrader tutorial](https://www.backtrader.com/docu/q.uickstart/quickstart/)

In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

In [2]:
import os
import sys
import yfinance as yf
import backtrader as bt
from datetime import datetime

import pandas as pd

In [3]:
# Add project folder to system path.
PROJECT_FOLDER = "/Users/wtai/Projects/Quantitative_trading/"
sys.path.append(PROJECT_FOLDER)
import src.utils as utils

In [4]:
# Download data
# tenb_df = utils.download_yf("TENB", "2y", "1d", "/Users/wtai/Projects/Quantitative_trading/data")
tenb_df = pd.read_csv("/Users/wtai/Projects/Quantitative_trading/data/TENB_20210527_20230526_1d.csv")
tenb_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2021-05-27 00:00:00-04:00,42.160000,42.299999,41.130001,42.049999,645700,0.0,0.0
1,2021-05-28 00:00:00-04:00,42.439999,42.540001,41.709999,41.799999,357900,0.0,0.0
2,2021-06-01 00:00:00-04:00,41.820000,41.990002,40.590000,41.230000,1022600,0.0,0.0
3,2021-06-02 00:00:00-04:00,41.169998,41.470001,40.660000,40.990002,570900,0.0,0.0
4,2021-06-03 00:00:00-04:00,40.630001,40.700001,39.779999,40.380001,516100,0.0,0.0
...,...,...,...,...,...,...,...,...
499,2023-05-22 00:00:00-04:00,39.790001,40.720001,39.240002,40.200001,1253100,0.0,0.0
500,2023-05-23 00:00:00-04:00,39.860001,40.220001,37.380001,37.400002,1189100,0.0,0.0
501,2023-05-24 00:00:00-04:00,36.959999,38.779999,36.540001,38.450001,1006500,0.0,0.0
502,2023-05-25 00:00:00-04:00,38.720001,38.939999,37.360001,37.630001,766800,0.0,0.0


In [32]:
# First strategy
# All strategies extends from the class bt.Strategy
class TestStrategy(bt.Strategy):
    """A test strategy that buys the stock when it has been falling
    in the past three days.
    
    Note
    ----
    * All strategies extends from the class bt.Strategy
    """
    # Parameters
    params = (
        ('exitbars', 5),
    )
    
    def log(self, txt:str, dt:datetime=None) -> None:
        """Print text to console."""
        dt = dt or self.datas[0].datetime.date(0)
        print("%s, %s" % (dt.isoformat(), txt))
    
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] data series.
        self.dataclose = self.datas[0].close
        # Keep track of the current pending order and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
    
    def notify_order(self, order):
        """The strategy is notified on the change of order status. The status change
        can be one of 'Submitted', 'Accepted', 'Completed', 'Canceled', 'Margin', 
        or 'Rejected'. This method works as a callback function. The order is passed 
        in as a parameter.

        Parameters
        ----------
        order: Any
            The order whose status is changed. This param is passed in from the system.
        
        Returns
        -------
        None

        Questions
        ---------
        * What is the 'Margin' order status?
        """
        if order.status in [order.Submitted, order.Accepted]:
            # Do nothing when a order is submitted to or accepted by the broker.
            return
        elif order.status in [order.Completed]:
            # Print out the order execution status for a completed buy or sell order.
            if order.isbuy():
                # Buy order executed
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm)
                )
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                # Sell order executed
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm: %.2f' % 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm)
                )
            # Keep a reference of the system clock when an order is executed.
            # The bar_executed can be understood as the elapsed periods
            # since the backtesting started.
            self.bar_executed = len(self)
            self.log("ORDER EXECUTED at %i-th bar" % self.bar_executed)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled/Margin/Rejected")
        # Set the pending order to none when the order status is Completed,
        # Canceled, Margin, or Rejected.
        self.order = None

    def notify_trade(self, trade):
        """Callback function which is called by the system when the backtesting finishes.
        
        Parameters
        ----------
        trade: Any

        Returns
        -------
        None
        """
        if not trade.isclosed:
            return
        self.log(
            "Operation profit, Gross %.2f, Net %.2f" %
            (trade.pnl, trade.pnlcomm)
        )

    def next(self):
        """This methods is called on each bar of the system clock. In the Backtrader 
        jargon a *bar* is a period - 1 minute, 1 hour, 1 day, 1 week or another time 
        period.
        
        Questions
        ---------
        * What is the system clock?
        """
        self.log("Close, %.2f" % self.dataclose[0])
        if self.order is not None:
            # Do nothing at the bar when there is a pending order.
            # The order can be a buy or a sell order.
            return
        
        if not self.position:
            # Only buy in when hold no shares of this stock.
            if self.dataclose[0] < self.dataclose[-1]:
                # Current close is less than previous close. Note that
                # the subscript 0, -1 and -2 are relative to the current
                # system clock.
                if self.dataclose[-1] < self.dataclose[-2]:
                    # Create a buy order using the closing pricing of
                    # the current system clock. Keep a reference of the
                    # order. A default buy order is a 'market buy order'
                    # - it buys in at the openning price of the next bar
                    # regardless what it is. Similarly, a default sell 
                    # order is a 'market sell order' and would be executed
                    # at the next bar openning price regardless what it is.
                    # In contrast, a limit order is only executed when the
                    # price hit the price set with the order.
                    self.log("BUY CREATE, %.2f" % self.dataclose[0])
                    self.order = self.buy()
        else:
            # Check if the exit criteria is met when hold shares.
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # Sell the stock after holding it for 5 periods since the last
                # recorded bar. Create a sell order using the closing price of
                # the current bar. Update the order reference.
                self.log("SELL CREATED. %.2f" % self.dataclose[0])
                self.order = self.sell()

Use backtrader to backtest a simple strategy.

In [33]:
# Data file
data_file = "TENB_20210527_20230526_1d.csv"
# Create Cerebro engine.
cerebro = bt.Cerebro()
# Add a strategy
cerebro.addstrategy(TestStrategy)
# Crate a data object from local CSV data downloaded from YahooFinance.
# The YahooFinanceCSVData does not comply with today's YahooFinance data.
# Hence use GenericCSVData instead.
data = bt.feeds.GenericCSVData(
    dataname=os.path.join(PROJECT_FOLDER, "data", data_file),
    # Datetime format
    dtformat='%Y-%m-%d %H:%M:%S%z',
    # Do not pass values before this date.
    fromdate=datetime(2021, 5, 27),
    # Do not pass values after this date.
    todate=datetime(2023, 5, 26),
    # Datetime column position
    datetime=0,
    # Time column not exist (-1)
    time=-1,
    # Open column position
    open=1,
    # High column position
    high=2,
    # Low column position
    low=3,
    # Close column position
    close=4,
    # Volume column position
    volume=5,
    # Reverse ordered
    reverse=False
)
# Add data object to the Cerebro engine.
cerebro.adddata(data)
# Set the initial cash value.
cerebro.broker.setcash(1e5)
# Sizer seems to be the amount of shares to buy or sell per order.
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
# Set the commision to be 0.1%. Degiro has a fixed 2$ commission per transaction for US stocks.
cerebro.broker.setcommission(commission=1e-3)
# Run backtesting
print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
cerebro.run()
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00
2021-05-27, Close, 42.05
2021-05-28, Close, 41.80
2021-06-01, Close, 41.23
2021-06-01, BUY CREATE, 41.23
2021-06-02, BUY EXECUTED, Price: 41.17, Cost: 411.70, Comm 0.41
2021-06-02, ORDER EXECUTED at 4-th bar
2021-06-02, Close, 40.99
2021-06-03, Close, 40.38
2021-06-04, Close, 41.78
2021-06-07, Close, 42.60
2021-06-08, Close, 42.96
2021-06-09, Close, 42.27
2021-06-09, SELL CREATED. 42.27
2021-06-10, SELL EXECUTED, Price: 42.43, Cost: 411.70, Comm: 0.42
2021-06-10, ORDER EXECUTED at 10-th bar
2021-06-10, Operation profit, Gross 12.60, Net 11.76
2021-06-10, Close, 42.99
2021-06-11, Close, 43.47
2021-06-14, Close, 43.88
2021-06-15, Close, 42.90
2021-06-16, Close, 42.90
2021-06-17, Close, 43.32
2021-06-18, Close, 43.80
2021-06-21, Close, 43.57
2021-06-22, Close, 45.02
2021-06-23, Close, 44.36
2021-06-24, Close, 43.50
2021-06-24, BUY CREATE, 43.50
2021-06-25, BUY EXECUTED, Price: 43.71, Cost: 437.10, Comm 0.44
2021-06-25, ORDER EXECUTED at 21-th bar
2021-0

In [22]:
%matplotlib inline
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]